In [2]:
!pip install langdetect -q
from langdetect import detect

import string
valid_chars=string.ascii_letters+string.digits+' '

import pandas as pd
import numpy as np

import cufflinks as cf
cf.go_offline()
from collections import defaultdict

from google.colab import files

def configure_plotly_browser_state():
    #This function is required to see Plotly charts in colab. Call this function in every cell where a Plotly chart gets drawn.
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.43.1.min.js?noext',
            },
          });
        </script>
        '''))

     |████████████████████████████████| 983kB 3.8MB/s 


In [ ]:
!pip install kaggle --upgrade -q
files.upload()

In [ ]:
book_data_train=pd.read_csv('train/book_data.csv')
book_data_test=pd.read_csv('test/book_data.csv')

In [ ]:

book_data_train.head()

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import time

from requests import get

#Sometimes not including the header results in a failed response
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8'}

BASE_URL='https://www.goodreads.com'
LIST_URL='https://www.goodreads.com/list/show/1.Best_Books_Ever'
#LIST_URL='https://www.goodreads.com/list/best_of_year/2018?id=119307.Best_books_of_2018'

books={'URL':[]}

#the no. of pages this list has
num_pages=557

for i in range(1, num_pages):
  #time.sleep(120) #make sure you give enough time between page loads to not avoid server overload
  print(f'Reading page {i}')
  list_page_url=f'{LIST_URL}&page={i}'
  list_page=get(list_page_url, headers=hdr)
  list_soup = BeautifulSoup(list_page.content, 'html.parser')
  book_table=list_soup.find('table', attrs={'class':'tableList js-dataTooltip'})
  rows=book_table.find_all('tr')
  books['URL']+=[BASE_URL+r.find('a', attrs={'class':'bookTitle'}).attrs['href'] for r in rows]

books_df=pd.DataFrame.from_dict(books)
books_df.to_csv('books.csv')

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import time
import os

from requests import get

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8'}

if not os.path.exists('book_data.csv'):
  book_data=pd.DataFrame(columns=[
    'image_url',
    'book_title',
    'book_authors',
    'book_rating',
    'book_rating_count',
    'book_review_count',
    'book_desc',
    'book_format',
    'book_edition',
    'book_pages',
    'book_isbn',
    'genres'
  ])

  book_data.to_csv('book_data.csv')

books=pd.read_csv('books.csv')
book_data=pd.read_csv('book_data.csv')

book_rows=[]
save_every=10

for i in range(len(book_data), len(books)):
    time.sleep(60)
    try:
        book_URL=books.at[i, 'URL']
        book_page=get(book_URL)
        book_soup=BeautifulSoup(book_page.content, 'html.parser')

        book=dict()

        #Save the URL of the image of the book cover to be downloaded later
        image_url=book_soup.find('img', attrs={'id':'coverImage'})
        if image_url:
            book['image_url']=image_url.attrs['src']
        else:
            book['image_url']=''

        #Title of the book
        book_title=book_soup.find('h1', attrs={'id':'bookTitle'})
        if book_title:
            book['book_title']=book_title.text.replace('\n','').strip()
        else:
            book['book_title']=''
        
        print(i, book['book_title'])

        #Author(s) of the book
        book['book_authors']='|'.join([a.find('span', attrs={'itemprop':'name'}).text for a in book_soup.find_all('a', attrs={'class':'authorName'})])
        
        #Rating given by users on goodreads
        book_rating=book_soup.find('span', attrs={'itemprop':'ratingValue'})
        if book_rating:
            book['book_rating']=book_rating.text.replace('\n','').strip()
        else:
            book['book_rating']=''
        book['book_rating_count']=book_soup.find('meta', attrs={'itemprop':'ratingCount'})['content']

        #No. of reviews for the book
        book['book_review_count']=book_soup.find('meta', attrs={'itemprop':'reviewCount'})['content']

        #A short description of the book, usually found on the back or inside cover of the book. Also called a blurb
        book_desc=book_soup.find('div', attrs={'class':'readable stacked'})
        if book_desc:
            book['book_desc']=book_desc.find_all('span')[-1].text
        else:
            book['book_desc']=''

        #Format of the book, e.g, paperback, hardcover, Kindle edition, etc.
        book_format=book_soup.find('div', attrs={'id':'details'}).find('span', attrs={'itemprop':'bookFormat'})
        if book_format:
            book['book_format']=book_format.text
        else:
            book['book_format']=''
        
        #Edition of the book
        book_edition=book_soup.find('div', attrs={'id':'details'}).find('span', attrs={'itemprop':'bookEdition'})
        if book_edition:
            book['book_edition']=book_edition.text
        else:
            book['book_edition']=''

        #No. of pages in the book
        book_pages=book_soup.find('div', attrs={'id':'details'}).find('span', attrs={'itemprop':'numberOfPages'})
        if book_pages:
            book['book_pages']=book_pages.text
        else:
            book['book_pages']=''
        
        #ISBN code of the book
        book_isbn=book_soup.find('div', attrs={'id':'bookDataBox'}).find('span', attrs={'itemprop':'isbn'})
        if book_isbn:
            book['book_isbn']=book_isbn.text
        else:
            book['book_isbn']=''

        #List of genres that the book belongs to. User supplied data.
        genres_list=book_soup.find_all('a', attrs={'class':'actionLinkLite bookPageGenreLink'})
        book['genres']='|'.join([i.text for i in genres_list])
        book_rows.append(book)

        if i%save_every==0:
            book_data.append(pd.DataFrame.from_dict(book_rows)).to_csv('book_data.csv', index=False)
            book_rows=[]
    except:
        book_data.append(pd.DataFrame.from_dict(book_rows)).to_csv('book_data.csv', index=False)
        book_rows=[]


In [ ]:
configure_plotly_browser_state()

def genre_count(genres):
    '''
    Returns the number of genres in the genre column of the dataframe
    Input: genre data
    Output: no. of genres
    '''
    try:
        return len(genres.split('|'))
    except:
        return 0


book_data_train['num_genres'] = book_data_train['genres'].apply(genre_count)
book_data_train['num_genres'].iplot(kind='histogram', xTitle='No. of genres', yTitle='Count', gridcolor='rgba(0,0,0,0)')

In [ ]:
genre_counts=defaultdict(int)
for i in book_data_train.index:
    g=book_data_train.at[i, 'genres']
    if type(g)==str:
        for j in g.split('|'):
            genre_counts[j]+=1

print(len(genre_counts))

In [ ]:

configure_plotly_browser_state()
genre_counts_df=pd.DataFrame.from_records(sorted(genre_counts.items(), key=lambda x:x[1], reverse=True), columns=['genre', 'count'])
genre_counts_df[:50].iplot(
    kind='bar', 
    x='genre', 
    y='count',
    margin={
        'b':175
    },
    gridcolor='rgba(0,0,0,0)'

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from os.path import exists

def plot_book_covers(n):
    '''
    Prints a specified no. of book covers and their corresponding genres. Helps to examine if there is any pattern in the book covers
    Input: no. of book covers to display
    Output: None
    '''
    indices=list(np.random.randint(min(book_data_train.index), max(book_data_train.index), size=n))
    fig=plt.figure(figsize=(n*5, 10))
    for i in indices:
        if exists(f'train/images/{i}.jpg'):
            ax = fig.add_subplot(2, n, (indices.index(i)+1), xticks=[], yticks=[])
            ax.imshow(Image.open(f'train/images/{i}.jpg'))
            ax = fig.add_subplot(2, n, (indices.index(i)+1)+n, xticks=[], yticks=[])
            if type(book_data_train.at[i, 'genres'])==str:
                t='\n'.join(book_data_train.at[i, 'genres'].split('|'))
            else:
                t=''
            ax.text(0.5, 0.5, t, fontsize=15, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

In [ ]:
book_data_train['num_authors']=book_data_train['book_authors'].str.split('|').apply(len)
configure_plotly_browser_state()
book_data_train['num_authors'].iplot(kind='histogram', xTitle='No. of authors', yTitle='Count', gridcolor='rgba(0,0,0,0)')

In [ ]:
book_data_train[book_data_train['num_authors']>10].head()

In [ ]:
def remove_invalid(df):
    '''
    Removes records that have invalid descriptions from the dataframe
    Input: dataframe
    Output: Cleaned up dataframe
    '''
    invalid_desc_idxs=[]
    for i in df.index:
        try:
            a=detect(df.at[i,'book_desc'])
        except:
            invalid_desc_idxs.append(i)
    
    df=df.drop(index=invalid_desc_idxs)
    return df

book_data_train=remove_invalid(book_data_train)
book_data_train['lang']=book_data_train.book_desc.apply(detect)

#Downloading the list of languages to map the two-letter lang code to the language name
lang_lookup=pd.read_html('https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes')[1]
lang_lookup.drop(columns=[0], inplace=True)
lang_lookup.columns=lang_lookup.iloc[0]
lang_lookup=lang_lookup.reindex(lang_lookup.index.drop(0))
lang_lookup.rename(columns={'639-1': 'lang'}, inplace=True)

def get_language(lang):
    if lang in list(lang_lookup['lang']):
        return lang_lookup[lang_lookup['lang']==lang]['ISO language name'].values[0]
    else:
        return 'N/A'

book_data_train['language']=book_data_train['lang'].apply(get_language)

In [ ]:
configure_plotly_browser_state()
group_by_lang=book_data_train.groupby(['language'])['book_title'].count().reset_index().sort_values(by=['book_title'], ascending=False)
group_by_lang.iplot(
    kind='bar', 
    x='language', 
    y='book_title',
    margin={
        'b':200
    },
    gridcolor='rgba(0,0,0,0)'
)

In [ ]:
book_data_train[book_data_train['language']=='Tamil']

In [ ]:
!pip install langdetect -q
from langdetect import detect

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

import string
valid_chars=string.ascii_letters+string.digits+' '

import pandas as pd
import numpy as np

import cufflinks as cf

from google.colab import files

def configure_plotly_browser_state():
    #This function is required to see Plotly charts in colab. Call this function in every cell where a Plotly chart gets drawn.
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.43.1.min.js?noext',
            },
          });
        </script>
        '''))

In [ ]:
def genre_binarizer(genres):
    '''
    Analyzes the genres string passed as argument to check if fiction or nonfiction is present and returns a genre appropriately
    Inputs: the genres column data from the book data frame
    Returns:
    - fiction, if fiction is present
    - nonfiction, if nonfiction is present
    - neither, if neither is present
    '''
    genre_list=genres.lower().split('|')
    if 'fiction' in genre_list:
        return 'fiction'
    elif 'nonfiction' in genre_list:
        return 'nonfiction'
    else:
        return 'neither'

def eng_desc(df):
    '''
    Analyzes the book description in the data frame passed, and returns a data frame with non-English records removed using the langdetect package
    Inputs: data frame
    Returns: data frame with non-English description records removed
    '''
    invalid_desc_idxs=[]
    for i in df.index:
        try:
            a=detect(df.at[i,'book_desc'])
        except:
            invalid_desc_idxs.append(i)
    
    print('Removing these records as the descriptions are invalid:',invalid_desc_idxs)
    df=df.drop(index=invalid_desc_idxs)
    df['lang']=df.book_desc.apply(detect)
    df=df[df.lang=='en']
    return df

def add_space_case(desc):
    '''
    Analyzes the book description passed and inserts spaces where a lowercase letter is followed immediately by an uppercase letter.
    Inputs: book description
    Returns: Modified book description'''
    upd_desc=''
    
    for i in range(len(desc)-1):
        upd_desc+=desc[i]
        if desc[i] in string.ascii_lowercase and desc[i+1] in string.ascii_uppercase:
            upd_desc+=' '
    
    upd_desc+=desc[-1]
    return upd_desc  

def remove_punctuation(desc):
    '''
    Modifies the book description passed to 
    - insert spaces in place of punctuations
    - join apostrophe words to their parent words and 
    - insert spaces where lowercase is followed by uppercase
    Inputs: book description
    Returns: modified book description
    '''
    desc=add_space_case(desc)
    apostrophe_words=['m', 're', 've', 'll', 't', 's', 'd']
    
    desc=desc.lower()
        
    desc=''.join([c if c in valid_chars else ' ' for c in desc])
    
    for a in apostrophe_words:
        desc=desc.replace(' '+a+' ', a+' ')

    return desc

        
def df_cleaner(df):
    '''
    Takes in a dataframe and performs the following steps to clean up:
    - Removes records with null genres and descriptions
    - Removes records where genre is neither fiction nor nonfiction
    - Removes records where the description is non-English
    - Removes punctuations from the description
    - Resets the index
    Inputs: dataframe Returns: cleaned-up dataframe
    '''
    print("No. of records            :", len(df))
    
    df=df[df.genres.notnull()]
    print("After removing null genres:", len(df))
    
    df=df[df.book_desc.notnull()]
    print("After removing null descs :", len(df))
    
    df=df[df.book_desc.str.strip().apply(len)>0]
    print("After removing zero descs :", len(df))
    
    df['binary_genre']=df.genres.apply(genre_binarizer)
    df=df[df.binary_genre!='neither']
    print("Only fiction or nonfiction:", len(df))
    
    df=eng_desc(df)
    print("After removing non-English:", len(df))
    
    df['clean_desc']=df.book_desc.apply(remove_punctuation)
    
    df.reset_index(drop=True, inplace=True)
    print('---------------------------')
    return df

In [ ]:
#Calculating the no. of words in the book descriptions and storing them in a dataframe - this enables the use of cufflinks
len_df=pd.DataFrame()
desc_lengths=[len(i.split()) for i in book_data_train.clean_desc] + [len(i.split()) for i in book_data_test.clean_desc]
len_df['desc_lengths']=desc_lengths

cf.go_offline()

#This function needs to be called to draw Plotly charts in Colab
configure_plotly_browser_state()


#Binning the lengths (no. of words) and calculating a cumsum to figure the 80% mark
len_df_bins=len_df.desc_lengths.value_counts(bins=100, normalize=True).reset_index().sort_values(by=['index'])
len_df_bins['cumulative']=len_df_bins.desc_lengths.cumsum()
len_df_bins['index']=len_df_bins['index'].astype('str')
len_df_bins.iplot(kind='bar', x='index', y='cumulative')

In [ ]:
vocabulary=set() #unique list of all words

def add_to_vocab(df, vocabulary):
    for i in df.clean_desc:
        for word in i.split():
            vocabulary.add(word)
    return vocabulary

vocabulary=add_to_vocab(book_data_train, vocabulary)
vocabulary=add_to_vocab(book_data_test, vocabulary)

#This dictionary represents the mapping from word to token. Using token+1 to skip 0, since 0 will be used for padding descriptions with less than 200 words
vocab_dict={word: token+1 for token, word in enumerate(list(vocabulary))}

#This dictionary represents the mapping from token to word
token_dict={token+1: word for token, word in enumerate(list(vocabulary))}

assert token_dict[1]==token_dict[vocab_dict[token_dict[1]]]

In [ ]:
def tokenizer(desc, vocab_dict, max_desc_length):
    '''
    Function to tokenize descriptions
    Inputs:
    - desc, description
    - vocab_dict, dictionary mapping words to their corresponding tokens
    - max_desc_length, used for pre-padding the descriptions where the no. of words is less than this number
    Returns:
    List of length max_desc_length, pre-padded with zeroes if the desc length was less than max_desc_length
    '''
    a=[vocab_dict[i] if i in vocab_dict else 0 for i in desc.split()]
    b=[0] * max_desc_length
    if len(a)<max_desc_length:
        return np.asarray(b[:max_desc_length-len(a)]+a).squeeze()
    else:
        return np.asarray(a[:max_desc_length]).squeeze()

book_data_train['desc_tokens']=book_data_train.clean_desc.apply(tokenizer, args=(vocab_dict, max_desc_length))
book_data_test['desc_tokens']=book_data_test.clean_desc.apply(tokenizer, args=(vocab_dict, max_desc_length))

In [ ]:
model = Sequential()
model.add(Embedding(len(vocabulary)+1, output_dim=200, input_length=max_desc_length))
model.add(LSTM(200, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

model.fit(x_train, 
          y_train,
          validation_data=(x_val, y_val),
          batch_size=128, 
          epochs=2)
score = model.evaluate(x_test, y_test, batch_size=16)

In [ ]:
from bs4 import BeautifulSoup
import urllib.request

from requests import get

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
         'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
         'Referer': 'https://cssspritegenerator.com',
         'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
         'Accept-Encoding': 'none',
         'Accept-Language': 'en-US,en;q=0.8'}

In [ ]:
book_page=get(test_url)
book_soup=BeautifulSoup(book_page.content, 'html.parser')

book_desc=book_soup.find('div', attrs={'class':'readable stacked'})
if book_desc:
    test_string=book_desc.find_all('span')[-1].text
else:
    test_string=''

if test_string=='':
    print('Cannot read book description from web page')
else:
    a=tokenizer(remove_punctuation(test_string), vocab_dict, max_desc_length)
    a=np.reshape(a, (1,max_desc_length))
    
    output=model.predict(a, batch_size=1)

    #test_h = tuple([each.data for each in test_h])

    #output, _ = model(a, test_h)

    pred = np.asscalar((output>0.5)*1)

    print(classes[pred])

In [ ]:
import pandas as pd
import wget
import os

book_data=pd.read_csv('book_data.csv')
PATH='C:\\Python\\Python37-32\\Scripts\\code\\images\\'
files=os.listdir(PATH)
n=0
if len(files)>0:
    n=max([int(f[:-4]) for f in os.listdir(PATH)])+1

for i in range(n, len(book_data)):
    url=book_data.at[i, 'image_url']
    filename=f'{i}.jpg'
    if not pd.isna(url):
        wget.download(url, PATH+filename)
    if i%100==0:
        print(i)